In [1]:
#import xgboost as xgb
import pandas as pd
import numpy as np
import math as ma
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import linregress

df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

all_df_list = [df_train,df_test]

#for later
#scores = -1 * cross_val_score(my_pipeline, X, y,
#                              cv=5,
#                              scoring='neg_mean_absolute_error')

def makeDict(df,label):
    strcat_dict = {}
    for i,row in df.iterrows():
        strcat_dict[row[label]] = i 
    return strcat_dict

def makeOrdinal(df,label,showDict):
    filtered = df.sort_values([label], ascending = [True])
    df_filtered = filtered.groupby(label).first().reset_index()
    strcat_dict = {}
    
    for i,row in df_filtered.iterrows():
        strcat_dict[row[label]] = i 
    
    if showDict:
        print(strcat_dict)
    
    for j,row in df.iterrows():
        df.at[j,label] = strcat_dict.get(row[label])
    
    return df

df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [2]:
categorical_cols = ["MSZoning","Street","Alley","LotShape","LandContour","LotConfig",\
                    "LandSlope","Neighborhood","Condition1","Condition2","BldgType",\
                    "HouseStyle","RoofStyle","RoofMatl","Exterior1st","Exterior2nd",\
                    "MasVnrType","ExterQual","ExterCond","Foundation","BsmtQual",\
                    "BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2",\
                    "Heating","HeatingQC","CentralAir","Electrical","KitchenQual",\
                    "Functional","FireplaceQu","GarageType","GarageFinish",\
                    "GarageQual","GarageCond","PavedDrive","Fence",\
                    "MiscFeature","SaleType","SaleCondition"]

for column in categorical_cols:
    for df_single in all_df_list:
        makeOrdinal(df_single,column,True)
df_train.head()

{'C (all)': 0, 'FV': 1, 'RH': 2, 'RL': 3, 'RM': 4}
{'C (all)': 0, 'FV': 1, 'RH': 2, 'RL': 3, 'RM': 4}
{'Grvl': 0, 'Pave': 1}
{'Grvl': 0, 'Pave': 1}
{'Grvl': 0, 'Pave': 1}
{'Grvl': 0, 'Pave': 1}
{'IR1': 0, 'IR2': 1, 'IR3': 2, 'Reg': 3}
{'IR1': 0, 'IR2': 1, 'IR3': 2, 'Reg': 3}
{'Bnk': 0, 'HLS': 1, 'Low': 2, 'Lvl': 3}
{'Bnk': 0, 'HLS': 1, 'Low': 2, 'Lvl': 3}
{'Corner': 0, 'CulDSac': 1, 'FR2': 2, 'FR3': 3, 'Inside': 4}
{'Corner': 0, 'CulDSac': 1, 'FR2': 2, 'FR3': 3, 'Inside': 4}
{'Gtl': 0, 'Mod': 1, 'Sev': 2}
{'Gtl': 0, 'Mod': 1, 'Sev': 2}
{'Blmngtn': 0, 'Blueste': 1, 'BrDale': 2, 'BrkSide': 3, 'ClearCr': 4, 'CollgCr': 5, 'Crawfor': 6, 'Edwards': 7, 'Gilbert': 8, 'IDOTRR': 9, 'MeadowV': 10, 'Mitchel': 11, 'NAmes': 12, 'NPkVill': 13, 'NWAmes': 14, 'NoRidge': 15, 'NridgHt': 16, 'OldTown': 17, 'SWISU': 18, 'Sawyer': 19, 'SawyerW': 20, 'Somerst': 21, 'StoneBr': 22, 'Timber': 23, 'Veenker': 24}
{'Blmngtn': 0, 'Blueste': 1, 'BrDale': 2, 'BrkSide': 3, 'ClearCr': 4, 'CollgCr': 5, 'Crawfor': 6, 'Ed

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,3,65.0,8450,1,None,3,3,AllPub,...,0,NaN,None,None,0,2,2008,8,4,208500
1,2,20,3,80.0,9600,1,None,3,3,AllPub,...,0,NaN,None,None,0,5,2007,8,4,181500
2,3,60,3,68.0,11250,1,None,0,3,AllPub,...,0,NaN,None,None,0,9,2008,8,4,223500
3,4,70,3,60.0,9550,1,None,0,3,AllPub,...,0,NaN,None,None,0,2,2006,8,0,140000
4,5,60,3,84.0,14260,1,None,0,3,AllPub,...,0,NaN,None,None,0,12,2008,8,4,250000


In [3]:
#drop columns that are too sparse to be of any statistical significance
print("Before", df_train.shape, df_test.shape, all_df_list[0].shape, all_df_list[1].shape)

df_train = df_train.drop(['Utilities','PoolArea','PoolQC'], axis=1)
df_test = df_test.drop(['Utilities','PoolArea','PoolQC'], axis=1)
all_df_list = [df_train,df_test]

print("After", df_train.shape, df_test.shape, all_df_list[0].shape, all_df_list[1].shape)

Before (1460, 81) (1459, 80) (1460, 81) (1459, 80)
After (1460, 78) (1459, 77) (1460, 78) (1459, 77)


In [9]:
#Analysis only
#only select the int or float columns
int_cols = [cname for cname in df_train.columns if df_train[cname].dtype in ['int64']]
float_cols = [cname for cname in df_train.columns if df_train[cname].dtype in ['float64']]

sale_values = df_train['SalePrice']
primary_correlations = []
for val in int_cols:
    primary_correlations.append((val,linregress(df_train[val],sale_values).rvalue))

df_float_cor = df_train.copy()

for i,row in df_float_cor[float_cols].iterrows():
    if ma.isnan(row['LotFrontage']):
        df_float_cor.at[i,'LotFrontage'] = 0 #nan means no frontage, so 0 is sensible
    if ma.isnan(row['MasVnrArea']):
        df_float_cor.at[i,'MasVnrArea'] = 0 #nan means no area, so 0 is sensible
    if ma.isnan(row['GarageYrBlt']):
        df_float_cor.at[i,'GarageYrBlt'] = df_train['GarageYrBlt'].mean() #nan means unknown, so use the mean

for val in float_cols:
    primary_correlations.append((val,linregress(df_float_cor[val],sale_values).rvalue))
        
print("number of numeric columns:",len(primary_correlations))
correldf = pd.DataFrame(primary_correlations, columns=["column name","r value"])
correldf.sort_values(by='r value',ascending=True)

number of numeric columns: 37


,column name,r value
20,KitchenAbvGr,-0.135907
27,EnclosedPorch,-0.128578
1,MSSubClass,-0.084284
4,OverallCond,-0.077856
32,YrSold,-0.028923
13,LowQualFinSF,-0.025606
0,Id,-0.021917
30,MiscVal,-0.021190
16,BsmtHalfBath,-0.016844
8,BsmtFinSF2,-0.011378
